In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

/Users/tdx/anaconda2/envs/ds-env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/tdx/anaconda2/envs/ds-env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
%matplotlib inline

/Users/tdx/anaconda2/envs/ds-env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape )

tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')

In [4]:
tf_idf

<59071x547979 sparse matrix of type '<class 'numpy.float64'>'
	with 10379283 stored elements in Compressed Sparse Row format>

In [5]:
tf_idf = normalize(tf_idf)

In [6]:
%ls

Modeling text data with a hierarchy of clusters.ipynb
_167fc84b78dc145609e919983b475aaa_people_wiki.csv.zip
_395a4cfb2299d1655f1ef6bf6cc4f71b_people_wiki_tf_idf.npz.zip
_96eadbec4d43a0b0870dde27d0652fb2_people_wiki_map_index_to_word.gl.zip
_96eadbec4d43a0b0870dde27d0652fb2_people_wiki_map_index_to_word.json.zip
_a20552cf806c54b71b3b87cb593b9968_people_wiki.gl.zip
people_wiki.csv
people_wiki.gl/
people_wiki_map_index_to_word.gl/
people_wiki_map_index_to_word.json
people_wiki_tf_idf.npz


In [7]:
map_index_to_word_series = pd.read_json('people_wiki_map_index_to_word.json', typ='series')

In [8]:
map_index_to_word_series

biennials                                                                                                       522004
lb915                                                                                                           116290
shatzky                                                                                                         127647
woode                                                                                                           174106
damfunk                                                                                                         133206
nualart                                                                                                         153444
hatefillot                                                                                                      164111
missionborn                                                                                                     261765
yeardescribed                                   

## Bipartition the Wikipedia dataset using k-means

In [55]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    '''cluster: should be dictionary containing the following keys
    * dataframe: original dataframe
    * matrix: same data, in matrix format
    * centroid: centroid for this particular cluster
    '''
    
    data_matrix = cluster['matrix']
    dataframe = cluster['dataframe']
    
    print (data_matrix.shape)
    print (dataframe.shape)
    
    # Run k-means on the data matrix with k=2. We use scikit-learn here to simplify workflow.
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs,
                         random_state=seed, n_jobs=-1, verbose=1)
    kmeans_model.fit(data_matrix)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    print ('Done K-means')
    
    # Divide the data matrix into two parts using the cluster assignments.
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], \
    data_matrix[cluster_assignment==1]
    
    # Divide the dataframe into two parts, again using the cluster assignments
    cluster_assigment_sa = pd.DataFrame(cluster_assignment)
    dataframe_left_child, dataframe_right_child = dataframe[cluster_assigment_sa[0]==0], \
    dataframe[cluster_assigment_sa[0]==1]
    
    # Package relevant variables for the child clusters
    cluster_left_child = {
        'matrix': data_matrix_left_child,
        'dataframe': dataframe_left_child,
        'centroid': centroids[0]
    }
    
    cluster_right_child = {
        'matrix': data_matrix_right_child,
        'dataframe': dataframe_right_child,
        'centroid': centroids[1]
    }
    
    return (cluster_left_child, cluster_right_child)

In [47]:
%ls

Modeling text data with a hierarchy of clusters.ipynb
_167fc84b78dc145609e919983b475aaa_people_wiki.csv.zip
_395a4cfb2299d1655f1ef6bf6cc4f71b_people_wiki_tf_idf.npz.zip
_96eadbec4d43a0b0870dde27d0652fb2_people_wiki_map_index_to_word.gl.zip
_96eadbec4d43a0b0870dde27d0652fb2_people_wiki_map_index_to_word.json.zip
_a20552cf806c54b71b3b87cb593b9968_people_wiki.gl.zip
people_wiki.csv
people_wiki.gl/
people_wiki_map_index_to_word.gl/
people_wiki_map_index_to_word.json
people_wiki_tf_idf.npz


In [52]:
wiki = pd.read_csv('people_wiki.csv')

In [53]:
len(wiki)

59071

In [36]:
wiki_sub = wiki[:1000]

In [37]:
tf_idf_sub = tf_idf[:1000]

In [58]:
import time

In [59]:
start = time.time()
wiki_data = {'matrix': tf_idf, 'dataframe': wiki} # no 'centroid' for the root cluster
left_child, right_child = bipartition(wiki_data, maxiter=100, num_runs=6, seed=1)
print (time.time() - start)

(59071, 547979)
(59071, 3)
Done K-means
2824.11634516716


In [71]:
assignment_tmp = np.zeros(len(wiki))

In [73]:
assignment_tmp[1] = 1

In [77]:
assignment_tmp_df = pd.DataFrame(assignment_tmp)

In [82]:
assignment_tmp_df

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [83]:
wiki[assignment_tmp_df[0] == 0]

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...
5,<http://dbpedia.org/resource/Sam_Henderson>,Sam Henderson,sam henderson born october 18 1969 is an ameri...
6,<http://dbpedia.org/resource/Aaron_LaCrate>,Aaron LaCrate,aaron lacrate is an american music producer re...
7,<http://dbpedia.org/resource/Trevor_Ferguson>,Trevor Ferguson,trevor ferguson aka john farrow born 11 novemb...
8,<http://dbpedia.org/resource/Grant_Nelson>,Grant Nelson,grant nelson born 27 april 1971 in london also...
9,<http://dbpedia.org/resource/Cathy_Caruth>,Cathy Caruth,cathy caruth born 1955 is frank h t rhodes pro...
10,<http://dbpedia.org/resource/Sophie_Crumb>,Sophie Crumb,sophia violet sophie crumb born september 27 1...


In [76]:
tf_idf[assignment_tmp == 1].toarray()

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.94441209e-05, 7.61118518e-05, 2.07510998e-06]])

In [86]:
left_child['matrix']

<11510x547979 sparse matrix of type '<class 'numpy.float64'>'
	with 1885831 stored elements in Compressed Sparse Row format>

In [70]:
left_child['matrix'][0].toarray()

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.61305464e-04, 3.83085501e-05, 3.52499352e-05]])

In [117]:
distances = pairwise_distances(tf_idf, [left_child['centroid'], right_child['centroid']], metric='euclidean')

In [118]:
assignment = distances.argmin(axis=1)

In [119]:
distances

array([[0.97235187, 0.99832177],
       [1.01011505, 0.99938693],
       [1.00223891, 0.99429088],
       ...,
       [0.99121121, 0.99940234],
       [0.9788349 , 0.99641063],
       [1.00705551, 0.99743785]])

In [120]:
assignment

array([0, 1, 1, ..., 0, 0, 1])

In [124]:
(assignment == 0).sum()

11510

In [122]:
left_child['matrix'].shape[0]

11510

In [134]:
pd.Series(assignment)

0        0
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       0
18       1
19       1
20       1
21       0
22       0
23       1
24       1
25       0
26       1
27       1
28       0
29       1
        ..
59041    1
59042    1
59043    1
59044    1
59045    1
59046    1
59047    1
59048    1
59049    1
59050    1
59051    0
59052    0
59053    1
59054    1
59055    1
59056    1
59057    1
59058    1
59059    1
59060    1
59061    1
59062    1
59063    1
59064    1
59065    0
59066    1
59067    1
59068    0
59069    0
59070    1
Length: 59071, dtype: int64

In [142]:
type(wiki['name'] == 'Digby Morrell')

pandas.core.series.Series

In [146]:
left_child['dataframe'] = wiki[pd.Series(assignment) == 0]

In [147]:
right_child['dataframe'] = wiki[pd.Series(assignment) == 1]

## Visualize the bipartition

In [ ]:
len(wiki)

In [173]:
def display_single_tf_idf_cluster(cluster, map_index_to_word):
    '''map_index_to_word: Dataframe specifying the mapping between words and column indices
    '''
    
    wiki_subset = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid = cluster['centroid']
    
    # Print top 5 words with largest TF-IDF weights in the cluster
    idx = centroid.argsort()[::-1]
    
    for i in range(5):
        print ('{0:s}:{1:.3f}'.format(map_index_to_word[idx[i]], centroid[idx[i]]))
    
    print ('')
    
    # Compute distances from the centroid to all data points in the cluster
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    
    # Compute nearest neighbors of the centroid within the cluster
    nearest_neighbors = distances.argsort()
    
    # For 8 nearest neighbors, print the title as well as first 180 characters of text
    # Wrap the text at 80-character mark
    for i in range(8):
        text = ' '.join(wiki_subset.iloc[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print ('* {0:50s} {1:.5f}\n {2:s}\n {3:s}'.format(
            wiki_subset.iloc[nearest_neighbors[i]]['name'],
            distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
        
    
    print('')

In [176]:
import string

In [178]:
'a'.split(3)

TypeError: must be str or None, not int

In [182]:
string.split?

Object `string.split` not found.


In [181]:
'adfdfdf df fdf df '.split(None, 2)

['adfdfdf', 'df', 'fdf df ']

In [ ]:
'a'.split

In [162]:
wiki.iloc[[0,1]]['text']

0    digby morrell born 10 october 1979 is a former...
1    alfred j lewy aka sandy lewy graduated from un...
Name: text, dtype: object

In [149]:
map_index_to_word_series[261765]

301032

In [150]:
dict([('a', 1), ('b',2)])

{'a': 1, 'b': 2}

In [151]:
for u in map_index_to_word_series[:10].keys():
    print (u, map_index_to_word_series[u])

biennials 522004
lb915 116290
shatzky 127647
woode 174106
damfunk 133206
nualart 153444
hatefillot 164111
missionborn 261765
yeardescribed 161075
theoryhe 521685


In [101]:
map_index_to_word_series['nualart']

153444

In [104]:
map_index_to_word = dict([(map_index_to_word_series[word], word) for word in map_index_to_word_series.keys()])

In [107]:
map_index_to_word[174106]

'woode'

In [165]:
display_single_tf_idf_cluster(left_child, map_index_to_word)
display_single_tf_idf_cluster(right_child, map_index_to_word)

league:0.040
season:0.036
team:0.029
football:0.029
played:0.028

* Todd Williams                                      0.95468
 todd michael williams born february 13 1971 in syracuse new york is a former major league 
 baseball relief pitcher he attended east syracuseminoa high school
* Gord Sherven                                       0.95622
 gordon r sherven born august 21 1963 in gravelbourg saskatchewan and raised in mankota sas
 katchewan is a retired canadian professional ice hockey forward who played
* Justin Knoedler                                    0.95639
 justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
 baseball catcherknoedler was originally drafted by the st louis cardinals
* Chris Day                                          0.95648
 christopher nicholas chris day born 28 july 1975 is an english professional footballer who
  plays as a goalkeeper for stevenageday started his career at tottenham
* Tony Smith (footballer, born

## Perform recursive bipartitioning

In [166]:
athletes = left_child
non_athletes = right_child

In [167]:
# Bipartition the cluster of athletes
left_child_athletes, right_child_athletes = bipartition(athletes, maxiter=100, num_runs=6, seed=1)

(11510, 547979)
(11510, 3)
Done K-means


In [170]:
display_single_tf_idf_cluster(left_child_athletes, map_index_to_word)

league:0.054
season:0.043
football:0.038
played:0.035
coach:0.030



KeyError: 3439

In [171]:
display_single_tf_idf_cluster(right_child_athletes, map_index_to_word)

championships:0.045
tour:0.043
championship:0.035
world:0.031
won:0.031



KeyError: 1863